In [14]:
import numpy as np
import random
import copy
import time
import tkinter as tk
from tkinter import messagebox, font
import threading


def create_board():
    return np.zeros((3, 3, 3, 3), dtype=int)

def create_small_boards_status():
    return np.zeros((3, 3), dtype=int)

def is_valid_move(board, small_boards_status, active_small_board, big_row, big_col, small_row, small_col):
    if active_small_board is not None:
        required_big_row, required_big_col = active_small_board
        if (big_row, big_col) != (required_big_row, required_big_col):
            if small_boards_status[required_big_row, required_big_col] == 0:
                return False

    if small_boards_status[big_row, big_col] != 0:
        return False

    return board[big_row, big_col, small_row, small_col] == 0

def get_valid_moves(board, small_boards_status, active_small_board):
    valid_moves = []
    
    if active_small_board is None or small_boards_status[active_small_board] != 0:
        for big_row in range(3):
            for big_col in range(3):
                if small_boards_status[big_row, big_col] == 0:
                    for small_row in range(3):
                        for small_col in range(3):
                            if board[big_row, big_col, small_row, small_col] == 0:
                                valid_moves.append((big_row, big_col, small_row, small_col))
    else:
        big_row, big_col = active_small_board
        for small_row in range(3):
            for small_col in range(3):
                if board[big_row, big_col, small_row, small_col] == 0:
                    valid_moves.append((big_row, big_col, small_row, small_col))
    
    return valid_moves

def make_move(board, small_boards_status, player, big_row, big_col, small_row, small_col):
    board[big_row, big_col, small_row, small_col] = player
    
    small_board_winner = check_small_board_winner(board[big_row, big_col])
    if small_board_winner:
        small_boards_status[big_row, big_col] = small_board_winner
    elif np.all(board[big_row, big_col] != 0):
        small_boards_status[big_row, big_col] = 3
        
    if small_boards_status[small_row, small_col] == 0:
        next_active_board = (small_row, small_col)
    else:
        next_active_board = None
    
    return next_active_board

def check_small_board_winner(small_board):
    # Check rows
    for row in range(3):
        if small_board[row, 0] != 0 and small_board[row, 0] == small_board[row, 1] == small_board[row, 2]:
            return small_board[row, 0]
    
    # Check columns
    for col in range(3):
        if small_board[0, col] != 0 and small_board[0, col] == small_board[1, col] == small_board[2, col]:
            return small_board[0, col]
    
    # Check diagonals
    if small_board[0, 0] != 0 and small_board[0, 0] == small_board[1, 1] == small_board[2, 2]:
        return small_board[0, 0]
    if small_board[0, 2] != 0 and small_board[0, 2] == small_board[1, 1] == small_board[2, 0]:
        return small_board[0, 2]
    
    return 0

def check_game_winner(small_boards_status):
    # Check rows
    for row in range(3):
        if small_boards_status[row, 0] in [1, 2] and small_boards_status[row, 0] == small_boards_status[row, 1] == small_boards_status[row, 2]:
            return small_boards_status[row, 0]
    
    # Check columns
    for col in range(3):
        if small_boards_status[0, col] in [1, 2] and small_boards_status[0, col] == small_boards_status[1, col] == small_boards_status[2, col]:
            return small_boards_status[0, col]
    
    # Check diagonals
    if small_boards_status[0, 0] in [1, 2] and small_boards_status[0, 0] == small_boards_status[1, 1] == small_boards_status[2, 2]:
        return small_boards_status[0, 0]
    if small_boards_status[0, 2] in [1, 2] and small_boards_status[0, 2] == small_boards_status[1, 1] == small_boards_status[2, 0]:
        return small_boards_status[0, 2]
    
    if np.all(small_boards_status != 0):
        return 3
    
    return 0

def check_winning_move(board, small_board, player):
    # Check rows
    for row in range(3):
        if sum(small_board[row] == player) == 2 and sum(small_board[row] == 0) == 1:
            col = np.where(small_board[row] == 0)[0][0]
            return (row, col)
    
    # Check columns
    for col in range(3):
        if sum(small_board[:, col] == player) == 2 and sum(small_board[:, col] == 0) == 1:
            row = np.where(small_board[:, col] == 0)[0][0]
            return (row, col)
    
    # Check diagonals
    diag1 = small_board[np.arange(3), np.arange(3)]
    if sum(diag1 == player) == 2 and sum(diag1 == 0) == 1:
        idx = np.where(diag1 == 0)[0][0]
        return (idx, idx)
    
    diag2 = small_board[np.arange(3), 2 - np.arange(3)]
    if sum(diag2 == player) == 2 and sum(diag2 == 0) == 1:
        idx = np.where(diag2 == 0)[0][0]
        return (idx, 2 - idx)
    
    return None

def check_blocking_move(board, small_board, player):
    opponent = 3 - player
    return check_winning_move(board, small_board, opponent)

def forward_checking(board, small_boards_status, active_small_board, player):
    domains = {}
    
    valid_moves = get_valid_moves(board, small_boards_status, active_small_board)
    for move in valid_moves:
        domains[move] = [player]
    
    return domains

def ac3(board, small_boards_status, active_small_board, domains, player):
    move_scores = {}
    opponent = 3 - player
    
    for move in domains.keys():
        big_row, big_col, small_row, small_col = move
        score = 0
        
        
        test_board = copy.deepcopy(board)
        test_board[big_row, big_col, small_row, small_col] = player
        
        
        if check_small_board_winner(test_board[big_row, big_col]) == player:
            score += 100
        
        
        test_board[big_row, big_col, small_row, small_col] = opponent
        if check_small_board_winner(test_board[big_row, big_col]) == opponent:
            score += 50
        
        
        test_board[big_row, big_col, small_row, small_col] = player
        
        # check if this sends opponent to a board they can win
        if small_boards_status[small_row, small_col] == 0:
            opponent_small_board = board[small_row, small_col]
            if check_winning_move(board, opponent_small_board, opponent):
                score -= 30
        
        # Check if this sends opponent to a full or already won board
        if small_boards_status[small_row, small_col] != 0:
            score += 10
        
        # prefer center positions in small boards
        if small_row == 1 and small_col == 1:
            score += 5
        
        # prefer corner positions in small boards
        if (small_row == 0 and small_col == 0) or \
           (small_row == 0 and small_col == 2) or \
           (small_row == 2 and small_col == 0) or \
           (small_row == 2 and small_col == 2):
            score += 3
            
        # Prefer moves in center board
        if big_row == 1 and big_col == 1:
            score += 7
            
        # Add randomness to avoid predictable play
        score += random.randint(0, 2)
        
        move_scores[move] = score
    
    return move_scores

def mrv_heuristic(move_scores):
    if not move_scores:
        return None
    
    best_score = max(move_scores.values())
    best_moves = [move for move, score in move_scores.items() if score == best_score]
    
    return random.choice(best_moves)

def evaluate_board(board, small_boards_status, player):
    score = 0
    opponent = 3 - player
    
    # Score for won boards
    for big_row in range(3):
        for big_col in range(3):
            if small_boards_status[big_row, big_col] == player:
                score += 10
            elif small_boards_status[big_row, big_col] == opponent:
                score -= 10
    
    # Check potential wins on the big board
    for i in range(3):
        # Check rows
        row_sum_player = sum(1 for j in range(3) if small_boards_status[i, j] == player)
        row_sum_opponent = sum(1 for j in range(3) if small_boards_status[i, j] == opponent)
        if row_sum_player > 0 and row_sum_opponent == 0:
            score += row_sum_player * 3
        if row_sum_opponent > 0 and row_sum_player == 0:
            score -= row_sum_opponent * 3
        
        # Check columns
        col_sum_player = sum(1 for j in range(3) if small_boards_status[j, i] == player)
        col_sum_opponent = sum(1 for j in range(3) if small_boards_status[j, i] == opponent)
        if col_sum_player > 0 and col_sum_opponent == 0:
            score += col_sum_player * 3
        if col_sum_opponent > 0 and col_sum_player == 0:
            score -= col_sum_opponent * 3
    
    # Check diagonals on the big board
    diag1_sum_player = sum(1 for i in range(3) if small_boards_status[i, i] == player)
    diag1_sum_opponent = sum(1 for i in range(3) if small_boards_status[i, i] == opponent)
    if diag1_sum_player > 0 and diag1_sum_opponent == 0:
        score += diag1_sum_player * 3
    if diag1_sum_opponent > 0 and diag1_sum_player == 0:
        score -= diag1_sum_opponent * 3
    
    diag2_sum_player = sum(1 for i in range(3) if small_boards_status[i, 2-i] == player)
    diag2_sum_opponent = sum(1 for i in range(3) if small_boards_status[i, 2-i] == opponent)
    if diag2_sum_player > 0 and diag2_sum_opponent == 0:
        score += diag2_sum_player * 3
    if diag2_sum_opponent > 0 and diag2_sum_player == 0:
        score -= diag2_sum_opponent * 3
    
    # Evaluate active small boards
    for big_row in range(3):
        for big_col in range(3):
            if small_boards_status[big_row, big_col] == 0:
                small_board = board[big_row, big_col]
                
                # Check rows
                for row in range(3):
                    row_player = sum(1 for col in range(3) if small_board[row, col] == player)
                    row_opponent = sum(1 for col in range(3) if small_board[row, col] == opponent)
                    if row_player > 0 and row_opponent == 0:
                        score += row_player
                    if row_opponent > 0 and row_player == 0:
                        score -= row_opponent
                
                # Check columns
                for col in range(3):
                    col_player = sum(1 for row in range(3) if small_board[row, col] == player)
                    col_opponent = sum(1 for row in range(3) if small_board[row, col] == opponent)
                    if col_player > 0 and col_opponent == 0:
                        score += col_player
                    if col_opponent > 0 and col_player == 0:
                        score -= col_opponent
                
                # Check diagonals
                diag1_player = sum(1 for i in range(3) if small_board[i, i] == player)
                diag1_opponent = sum(1 for i in range(3) if small_board[i, i] == opponent)
                if diag1_player > 0 and diag1_opponent == 0:
                    score += diag1_player
                if diag1_opponent > 0 and diag1_player == 0:
                    score -= diag1_opponent
                
                diag2_player = sum(1 for i in range(3) if small_board[i, 2-i] == player)
                diag2_opponent = sum(1 for i in range(3) if small_board[i, 2-i] == opponent)
                if diag2_player > 0 and diag2_opponent == 0:
                    score += diag2_player
                if diag2_opponent > 0 and diag2_player == 0:
                    score -= diag2_opponent
                
                # Bonus for center position
                if small_board[1, 1] == player:
                    score += 2
                elif small_board[1, 1] == opponent:
                    score -= 2
    
    return score

def minimax_csp(board, small_boards_status, active_small_board, player, depth=3, alpha=float("-inf"), beta=float("inf")):
    winner = check_game_winner(small_boards_status)
    if winner != 0:
        if winner == player:
            return None, 100
        elif winner == 3:
            return None, 0
        else:
            return None, -100
    
    if depth == 0:
        return None, evaluate_board(board, small_boards_status, player)
    
    valid_moves = get_valid_moves(board, small_boards_status, active_small_board)
    
    if player == 2:  
        max_eval = float("-inf")
        best_move = None
        
        random.shuffle(valid_moves)  
        
        for move in valid_moves:
            big_row, big_col, small_row, small_col = move
            
            new_board = copy.deepcopy(board)
            new_status = copy.deepcopy(small_boards_status)
            next_active = make_move(new_board, new_status, player, big_row, big_col, small_row, small_col)
            
            _, eval_score = minimax_csp(new_board, new_status, next_active, 3 - player, depth - 1, alpha, beta)
            
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
                
        return best_move, max_eval
    else:  
        min_eval = float("inf")
        best_move = None
        
        random.shuffle(valid_moves)  
        
        for move in valid_moves:
            big_row, big_col, small_row, small_col = move
            
            new_board = copy.deepcopy(board)
            new_status = copy.deepcopy(small_boards_status)
            next_active = make_move(new_board, new_status, player, big_row, big_col, small_row, small_col)
            
            _, eval_score = minimax_csp(new_board, new_status, next_active, 3 - player, depth - 1, alpha, beta)
            
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
                
        return best_move, min_eval

def find_tactical_move(board, small_boards_status, active_small_board, player):
    valid_moves = get_valid_moves(board, small_boards_status, active_small_board)
    
    #win a small board immediate
    for move in valid_moves:
        big_row, big_col, small_row, small_col = move
        test_board = copy.deepcopy(board)
        test_board[big_row, big_col, small_row, small_col] = player
        if check_small_board_winner(test_board[big_row, big_col]) == player:
            return move
    
    # block opponent from winning a small board
    opponent = 3 - player
    for move in valid_moves:
        big_row, big_col, small_row, small_col = move
        test_board = copy.deepcopy(board)
        test_board[big_row, big_col, small_row, small_col] = opponent
        if check_small_board_winner(test_board[big_row, big_col]) == opponent:
            return move
    
    # Third priority: create a fork (two potential winning moves)
    for move in valid_moves:
        big_row, big_col, small_row, small_col = move
        test_board = copy.deepcopy(board)
        test_board[big_row, big_col, small_row, small_col] = player
        small_board = test_board[big_row, big_col]
        
        winning_opportunities = 0
        
        # Check rows
        for row in range(3):
            if sum(small_board[row] == player) == 2 and sum(small_board[row] == 0) == 1:
                winning_opportunities += 1
        
        # Check columns
        for col in range(3):
            if sum(small_board[:, col] == player) == 2 and sum(small_board[:, col] == 0) == 1:
                winning_opportunities += 1
        
        # Check diagonals
        diag1 = small_board[np.arange(3), np.arange(3)]
        if sum(diag1 == player) == 2 and sum(diag1 == 0) == 1:
            winning_opportunities += 1
        
        diag2 = small_board[np.arange(3), 2 - np.arange(3)]
        if sum(diag2 == player) == 2 and sum(diag2 == 0) == 1:
            winning_opportunities += 1
        
        if winning_opportunities >= 2:
            return move
    
    return None

def ai_decide_move(board, small_boards_status, active_small_board, player):
    #  check for tactical moves
    tactical_move = find_tactical_move(board, small_boards_status, active_small_board, player)
    if tactical_move:
        return tactical_move
    

    move_count = np.sum(board != 0)
    
    depth = 2
    if move_count > 20:
        depth = 3
    if move_count > 40:
        depth = 4
    
    best_move, _ = minimax_csp(board, small_boards_status, active_small_board, player, depth=depth)
    
    if best_move:
        return best_move
    
    # Fallback to AC3 + heuristics
    domains = forward_checking(board, small_boards_status, active_small_board, player)
    move_scores = ac3(board, small_boards_status, active_small_board, domains, player)
    return mrv_heuristic(move_scores)

def count_moves(board):
    return np.sum(board != 0)

# Tkinter GUI Implementation
class UltimateTicTacToeGUI:
    def __init__(self, master):
        self.master = master
        master.title("Ultimate Tic-Tac-Toe")
        
        # Game state
        self.board = create_board()
        self.small_boards_status = create_small_boards_status()
        self.active_small_board = None
        self.current_player = 1  # X goes first
        self.game_over = False
        
        # Colors and fonts
        self.active_color = "#d6eaf8"  # Light blue
        self.x_color = "#f39c12"       # Orange
        self.o_color = "#e74c3c"       # Red
        self.draw_color = "#2ecc71"    # Green
        self.grid_color = "#34495e"    # Dark blue
        
        self.cell_font = font.Font(family="Arial", size=14, weight="bold")
        self.status_font = font.Font(family="Arial", size=12)
        self.title_font = font.Font(family="Arial", size=16, weight="bold")
        
        # Create main frames
        self.game_frame = tk.Frame(master)
        self.game_frame.pack(pady=10)
        
        self.status_frame = tk.Frame(master)
        self.status_frame.pack(pady=10, fill="x")
        
        # Create status label
        self.status_var = tk.StringVar()
        self.status_var.set("Game started. You are X, AI is O.")
        self.status_label = tk.Label(self.status_frame, textvariable=self.status_var, 
                                    font=self.status_font, wraplength=400)
        self.status_label.pack(pady=5)
        
        # Create board status display
        self.board_status_frame = tk.Frame(self.status_frame)
        self.board_status_frame.pack(pady=5)
        
        self.board_status_labels = {}
        for big_row in range(3):
            frame_row = tk.Frame(self.board_status_frame)
            frame_row.pack()
            for big_col in range(3):
                label = tk.Label(frame_row, text=f"[{big_row},{big_col}]: Active", 
                               font=self.status_font, width=15)
                label.pack(side=tk.LEFT, padx=5, pady=2)
                self.board_status_labels[(big_row, big_col)] = label
        
        # Create new game button
        self.new_game_btn = tk.Button(self.status_frame, text="New Game", 
                                     font=self.status_font, command=self.reset_game)
        self.new_game_btn.pack(pady=10)
        
        # Create the game grid
        self.buttons = {}
        
        # First create 3x3 frames for big boards
        self.big_frames = {}
        for big_row in range(3):
            for big_col in range(3):
                big_frame = tk.Frame(self.game_frame, bd=3, relief="raised")
                big_frame.grid(row=big_row, column=big_col, padx=3, pady=3)
                self.big_frames[(big_row, big_col)] = big_frame
                
                # Inside each big frame, create 3x3 buttons for small board
                for small_row in range(3):
                    for small_col in range(3):
                        btn = tk.Button(big_frame, text="", width=3, height=1, 
                                       font=self.cell_font,
                                       command=lambda br=big_row, bc=big_col, 
                                              sr=small_row, sc=small_col: 
                                              self.on_button_click(br, bc, sr, sc))
                        btn.grid(row=small_row, column=small_col, padx=1, pady=1)
                        self.buttons[(big_row, big_col, small_row, small_col)] = btn
        
        # Update the board display initially
        self.update_board()
    
    def update_board(self):
        for big_row in range(3):
            for big_col in range(3):
                big_frame = self.big_frames[(big_row, big_col)]
                
                # Set frame highlight based on active board
                if self.active_small_board == (big_row, big_col) and not self.game_over:
                    big_frame.config(bg=self.active_color)
                else:
                    big_frame.config(bg=self.master.cget('bg'))  # Default background
                
                # Update small board status label
                status = self.small_boards_status[big_row, big_col]
                status_text = "Active"
                fg_color = "black"
                
                if status == 1:
                    status_text = "X won"
                    fg_color = self.x_color
                elif status == 2:
                    status_text = "O won"
                    fg_color = self.o_color
                elif status == 3:
                    status_text = "Draw"
                    fg_color = self.draw_color
                
                label = self.board_status_labels[(big_row, big_col)]
                label.config(text=f"[{big_row},{big_col}]: {status_text}", fg=fg_color)
                
                # Update buttons in this small board
                for small_row in range(3):
                    for small_col in range(3):
                        btn = self.buttons[(big_row, big_col, small_row, small_col)]
                        cell_value = self.board[big_row, big_col, small_row, small_col]
                        
                        # Set button text
                        if cell_value == 1:
                            btn.config(text="X", fg=self.x_color)
                        elif cell_value == 2:
                            btn.config(text="O", fg=self.o_color)
                        else:
                            btn.config(text="")
                        
                        # Determine if button should be enabled
                        is_valid = is_valid_move(self.board, self.small_boards_status, 
                                              self.active_small_board, big_row, big_col, 
                                              small_row, small_col)
                        
                        if is_valid and not self.game_over and self.current_player == 1:
                            btn.config(state=tk.NORMAL)
                            if self.active_small_board == (big_row, big_col):
                                btn.config(bg=self.active_color)
                            else:
                                btn.config(bg="white")
                        else:
                            btn.config(state=tk.DISABLED)
                            if status == 1:  
                                btn.config(bg="#fef5e7")  
                            elif status == 2:  
                                btn.config(bg="#fadbd8")  
                            elif status == 3:  
                                btn.config(bg="#d5f5e3")  
                            else:
                                btn.config(bg="white")
    
    def on_button_click(self, big_row, big_col, small_row, small_col):
        if self.game_over or self.current_player != 1:
            return
        
        # make the move if valid
        if is_valid_move(self.board, self.small_boards_status, self.active_small_board, 
                       big_row, big_col, small_row, small_col):
            self.active_small_board = make_move(self.board, self.small_boards_status, 
                                             self.current_player, big_row, big_col, 
                                             small_row, small_col)
            self.current_player = 3 - self.current_player  # Switch player
            
            # Check if game is over
            winner = check_game_winner(self.small_boards_status)
            if winner != 0:
                if winner == 1:
                    self.status_var.set("You win!")
                elif winner == 2:
                    self.status_var.set("AI wins!")
                else:
                    self.status_var.set("It's a draw!")
                self.game_over = True
            else:
                # AI's turn
                self.status_var.set("AI is thinking...")
                self.update_board()
                self.master.update()  # Force update of UI
                
                # Start AI move in a separate thread to avoid UI freeze
                threading.Thread(target=self.ai_make_move).start()
            
            self.update_board()
    
    def ai_make_move(self):
        if self.game_over:
            return
        
        # Add a small delay to make AI's move visible
        time.sleep(0.5)
        
        # Get AI's move
        ai_move = ai_decide_move(self.board, self.small_boards_status, 
                               self.active_small_board, self.current_player)
        
        if ai_move:
            big_row, big_col, small_row, small_col = ai_move
            
            
            
            self.active_small_board = make_move(self.board, self.small_boards_status, 
                                             self.current_player, big_row, big_col, 
                                             small_row, small_col)
            self.current_player = 3 - self.current_player  # Switch player
            
            
            winner = check_game_winner(self.small_boards_status)
            if winner != 0:
                if winner == 1:
                    self.status_var.set("You win!")
                elif winner == 2:
                    self.status_var.set("AI wins!")
                else:
                    self.status_var.set("It's a draw!")
                self.game_over = True
            else:
                self.status_var.set("Your turn. You are X.")
        
        
        self.master.after(0, self.update_board)
    
    def reset_game(self):
        
        self.board = create_board()
        self.small_boards_status = create_small_boards_status()
        self.active_small_board = None
        self.current_player = 1  
        self.game_over = False
        
        
        self.status_var.set("New game started. You are X, AI is O.")
        
        
        self.update_board()

# main function to run the game
def main():
    root = tk.Tk()
    root.configure(bg="#f5f5f5")  
    
    
    window_width = 600
    window_height = 650
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    center_x = int(screen_width/2 - window_width/2)
    center_y = int(screen_height/2 - window_height/2)
    root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')
    
    
    game = UltimateTicTacToeGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()